In [1]:
import pandas as pd

# Define a function to load data and calculate its memory usage
def load_psv_and_report_memory(filename):
    data = pd.read_csv(filename, delimiter='|')
    memory = data.memory_usage(deep=True).sum() / (1024 ** 2)  # Convert bytes to megabytes
    print(f"Memory usage of {filename}: {memory:.2f} MB")
    return data

# Path to the data directory
data_dir = '../data/VIC_Address_Data/'

# Load data files with memory usage report
address_detail = load_psv_and_report_memory(data_dir + 'VIC_ADDRESS_DETAIL_psv.psv')
street_locality = load_psv_and_report_memory(data_dir + 'VIC_STREET_LOCALITY_psv.psv')
locality = load_psv_and_report_memory(data_dir + 'VIC_LOCALITY_psv.psv')
state = load_psv_and_report_memory(data_dir + 'VIC_STATE_psv.psv')
address_geocode = load_psv_and_report_memory(data_dir + 'VIC_ADDRESS_DEFAULT_GEOCODE_psv.psv')



C:\Users\ottend\AppData\Local\Temp\ipykernel_21284\3434893485.py:5: DtypeWarning: Columns (7,13,15,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, delimiter='|')


Memory usage of ../data/VIC_Address_Data/VIC_ADDRESS_DETAIL_psv.psv: 3949.19 MB
Memory usage of ../data/VIC_Address_Data/VIC_STREET_LOCALITY_psv.psv: 79.12 MB
Memory usage of ../data/VIC_Address_Data/VIC_LOCALITY_psv.psv: 0.78 MB
Memory usage of ../data/VIC_Address_Data/VIC_STATE_psv.psv: 0.00 MB
Memory usage of ../data/VIC_Address_Data/VIC_ADDRESS_DEFAULT_GEOCODE_psv.psv: 845.16 MB


In [2]:
address_detail.columns, street_locality.columns,locality.columns, state.columns,address_geocode.columns

(Index(['ADDRESS_DETAIL_PID', 'DATE_CREATED', 'DATE_LAST_MODIFIED',
        'DATE_RETIRED', 'BUILDING_NAME', 'LOT_NUMBER_PREFIX', 'LOT_NUMBER',
        'LOT_NUMBER_SUFFIX', 'FLAT_TYPE_CODE', 'FLAT_NUMBER_PREFIX',
        'FLAT_NUMBER', 'FLAT_NUMBER_SUFFIX', 'LEVEL_TYPE_CODE',
        'LEVEL_NUMBER_PREFIX', 'LEVEL_NUMBER', 'LEVEL_NUMBER_SUFFIX',
        'NUMBER_FIRST_PREFIX', 'NUMBER_FIRST', 'NUMBER_FIRST_SUFFIX',
        'NUMBER_LAST_PREFIX', 'NUMBER_LAST', 'NUMBER_LAST_SUFFIX',
        'STREET_LOCALITY_PID', 'LOCATION_DESCRIPTION', 'LOCALITY_PID',
        'ALIAS_PRINCIPAL', 'POSTCODE', 'PRIVATE_STREET', 'LEGAL_PARCEL_ID',
        'CONFIDENCE', 'ADDRESS_SITE_PID', 'LEVEL_GEOCODED_CODE', 'PROPERTY_PID',
        'GNAF_PROPERTY_PID', 'PRIMARY_SECONDARY'],
       dtype='object'),
 Index(['STREET_LOCALITY_PID', 'DATE_CREATED', 'DATE_RETIRED',
        'STREET_CLASS_CODE', 'STREET_NAME', 'STREET_TYPE_CODE',
        'STREET_SUFFIX_CODE', 'LOCALITY_PID', 'GNAF_STREET_PID',
        'GNAF_STREET_

In [3]:
import gc  # Import garbage collector interface

# Perform merges with explicit suffix handling
merged_data = address_detail.merge(street_locality, on='STREET_LOCALITY_PID', how='left', suffixes=('', '_sl'))
merged_data.drop(merged_data.filter(regex='_sl$').columns.tolist(), axis=1, inplace=True)
gc.collect()  # Explicitly call garbage collector

merged_data = merged_data.merge(locality, on='LOCALITY_PID', how='left', suffixes=('', '_loc'))
merged_data.drop(merged_data.filter(regex='_loc$').columns.tolist(), axis=1, inplace=True)
gc.collect()  # Explicitly call garbage collector

merged_data = merged_data.merge(state, on='STATE_PID', how='left', suffixes=('', '_st'))
merged_data.drop(merged_data.filter(regex='_st$').columns.tolist(), axis=1, inplace=True)
gc.collect()  # Explicitly call garbage collector

merged_data = merged_data.merge(address_geocode, on='ADDRESS_DETAIL_PID', how='left', suffixes=('', '_geo'))
merged_data.drop(merged_data.filter(regex='_geo$').columns.tolist(), axis=1, inplace=True)
gc.collect()  # Explicitly call garbage collector

# Filter by confidence and state abbreviation for Victoria
filtered_data = merged_data[(merged_data['CONFIDENCE'] > -1) & (merged_data['STATE_ABBREVIATION'] == 'VIC')]

del merged_data  # Delete the large merged_data object to free up memory
gc.collect()  # Explicitly call garbage collector after deleting the object


0

In [ ]:
# # Formatting the address
# def format_address(row):
#     # Format NUMBER_FIRST and FLAT_NUMBER as integers if not NaN
#     number_first = int(row['NUMBER_FIRST']) if pd.notna(row['NUMBER_FIRST']) else ''
#     flat_number = f"{int(row['FLAT_NUMBER'])}/" if pd.notna(row['FLAT_NUMBER']) else ''
    
#     # Assemble the full address using the formatted components
#     # Include the STREET_TYPE_CODE if it is not empty or NaN
#     street_type = row['STREET_TYPE_CODE'] if pd.notna(row['STREET_TYPE_CODE']) else ''
#     return f"{flat_number}{number_first} {row['STREET_NAME']} {street_type} {row['LOCALITY_NAME']} VIC {row['POSTCODE']}"

# # Apply the custom function to format the full address
# filtered_data['Full Address'] = filtered_data.apply(format_address, axis=1)

# # Select relevant columns and rename them for clarity
# required_columns = filtered_data[['ADDRESS_DETAIL_PID', 'Full Address', 'POSTCODE']]
# required_columns.rename(columns={'Full Address': 'Formatted Address', 'POSTCODE': 'Postcode'}, inplace=True)

# # Save the formatted data to a CSV file
# required_columns.to_csv(data_dir + 'filtered_victorian_addresses.csv', index=False)

# print("Data processing complete and saved to filtered_victorian_addresses.csv.")


C:\Users\ottend\AppData\Local\Temp\ipykernel_28252\3853007981.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Full Address'] = filtered_data.apply(format_address, axis=1)
C:\Users\ottend\AppData\Local\Temp\ipykernel_28252\3853007981.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_columns.rename(columns={'Full Address': 'Formatted Address', 'POSTCODE': 'Postcode'}, inplace=True)


Data processing complete and saved to filtered_victorian_addresses.csv.


In [7]:
# Define the list of postcodes of interest
postcodes = [
    3101, 3102, 3103, 3104, 3123, 3124, 3125, 3126, 3127, 3128, 
    3142, 3143, 3144, 3145, 3146, 3147, 3148, 3161, 3162, 3181, 
    3182, 3183, 3184, 3185, 3186, 3187, 3204
]

# Convert postcodes in dataframe to integers to ensure matching types
filtered_data['POSTCODE'] = filtered_data['POSTCODE'].astype(int)

# Filter the merged_data to only include rows with postcodes in the specified list
filtered_data = filtered_data[filtered_data['POSTCODE'].isin(postcodes)]

# Output the size of the filtered data to confirm the reduction
print(f"Filtered data contains {len(filtered_data)} records from specified postcodes.")


Filtered data contains 306375 records from specified postcodes.


In [9]:
import numpy as np

# Convert NaN values for flat number and number first to empty string or zero before converting to int
filtered_data['FLAT_NUMBER'] = filtered_data['FLAT_NUMBER'].fillna(0).astype(int).astype(str)
filtered_data['NUMBER_FIRST'] = filtered_data['NUMBER_FIRST'].fillna(0).astype(int).astype(str)

# Combine flat number and number first with conditions
filtered_data['FLAT_NUMBER'] = np.where(filtered_data['FLAT_NUMBER'] == '0', '', filtered_data['FLAT_NUMBER'] + '/')
filtered_data['STREET'] = filtered_data['FLAT_NUMBER'] + filtered_data['NUMBER_FIRST'] + ' ' + \
                          filtered_data['STREET_NAME'].fillna('') + ' ' + \
                          filtered_data['STREET_TYPE_CODE'].fillna('')

# Strip extra spaces from the STREET column
filtered_data['STREET'] = filtered_data['STREET'].str.strip()

# Assign other address components
filtered_data['CITY'] = filtered_data['LOCALITY_NAME']
filtered_data['STATE'] = 'VIC'  # Assuming all addresses are in Victoria
filtered_data['ZIP'] = filtered_data['POSTCODE'].astype(str)  # Ensure ZIP is string
filtered_data['LAT'] = filtered_data['LATITUDE'].fillna('')
filtered_data['LON'] = filtered_data['LONGITUDE'].fillna('')

# Select relevant columns and rename them for clarity
required_columns = filtered_data[['ADDRESS_DETAIL_PID', 'STREET', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON']]
required_columns.rename(columns={'ADDRESS_DETAIL_PID': 'Id', 'STREET': 'ADDRESS'}, inplace=True)

# Save the formatted data to a CSV file
required_columns.to_csv(data_dir + 'formatted_victorian_addresses.csv', index=False)

print("Data processing complete and saved to formatted_victorian_addresses.csv.")


C:\Users\ottend\AppData\Local\Temp\ipykernel_21284\2990155578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['FLAT_NUMBER'] = filtered_data['FLAT_NUMBER'].fillna(0).astype(int).astype(str)
C:\Users\ottend\AppData\Local\Temp\ipykernel_21284\2990155578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['NUMBER_FIRST'] = filtered_data['NUMBER_FIRST'].fillna(0).astype(int).astype(str)
C:\Users\ottend\AppData\Local\Temp\ipykernel_21284\2990155578.py:8: SettingWithCopyWarning: 
A

Data processing complete and saved to formatted_victorian_addresses.csv.


In [11]:
birthplaces = [
    # Major Australian Cities
    "Melbourne VIC AUS", "Sydney NSW AUS", "Brisbane QLD AUS", "Perth WA AUS", 
    "Adelaide SA AUS", "Canberra ACT AUS", "Hobart TAS AUS", "Darwin NT AUS",

    # Large Regional Australian Cities
    "Geelong VIC AUS", "Newcastle NSW AUS", "Wollongong NSW AUS", "Gold Coast QLD AUS", 
    "Townsville QLD AUS", "Cairns QLD AUS", "Toowoomba QLD AUS", "Ballarat VIC AUS", 
    "Bendigo VIC AUS", "Launceston TAS AUS", "Albury NSW AUS", "Mackay QLD AUS", 
    "Rockhampton QLD AUS", "Bunbury WA AUS", "Coffs Harbour NSW AUS", "Wagga Wagga NSW AUS",

    # Additional Australian Cities
    "Sunshine Coast QLD AUS", "Mildura VIC AUS", "Tamworth NSW AUS", "Shepparton VIC AUS",
    "Alice Springs NT AUS", "Burnie TAS AUS", "Port Macquarie NSW AUS", "Devonport TAS AUS",

    # UK - Common places of birth for Australian migrants
    "London UK", "Manchester UK", "Birmingham UK", "Glasgow UK", "Liverpool UK", 
    "Edinburgh UK", "Bristol UK", "Leeds UK", "Sheffield UK", "Nottingham UK",

    # India - Common places of birth for Indian migrants in Australia
    "New Delhi IND", "Mumbai IND", "Bangalore IND", "Chennai IND", "Hyderabad IND", 
    "Ahmedabad IND", "Kolkata IND", "Pune IND", "Jaipur IND", "Lucknow IND",

    # China - Common places of birth for Chinese migrants in Australia
    "Beijing CHN", "Shanghai CHN", "Guangzhou CHN", "Shenzhen CHN", "Chengdu CHN",
    "Tianjin CHN", "Hangzhou CHN", "Chongqing CHN", "Nanjing CHN", "Wuhan CHN",

    # Vietnam - Common places of birth for Vietnamese migrants in Australia
    "Ho Chi Minh City VNM", "Hanoi VNM", "Da Nang VNM", "Hai Phong VNM",

    # New Zealand - Common places of birth for Kiwi migrants in Australia
    "Auckland NZ", "Wellington NZ", "Christchurch NZ", "Hamilton NZ", "Dunedin NZ",

    # Additional Common Birthplaces for Australian Residents
    "Rome ITA", "Athens GRC", "Manila PHL", "Johannesburg ZAF", "Beirut LBN"
]


In [13]:

def load_and_modify(filepath):
    # Load the patient data
    patients = pd.read_csv(filepath)

    # Sample addresses randomly
    sampled_addresses = required_columns.sample(n=len(patients), replace=True).reset_index(drop=True)

    # Replace address-related columns and set COUNTY and FIPS to empty
    address_columns = ['ADDRESS', 'CITY', 'STATE', 'ZIP', 'LAT', 'LON']
    patients[address_columns] = sampled_addresses[address_columns]
    patients['COUNTY'] = ""
    patients['FIPS'] = ""

    # Generate random Medicare-like numbers: format 4 5 1 (e.g., 1234 56789 1)
    patients['SSN'] = np.random.randint(1000, 10000, size=len(patients)).astype(str) + ' ' + \
                      np.random.randint(10000, 100000, size=len(patients)).astype(str) + ' ' + \
                      np.random.randint(1, 10, size=len(patients)).astype(str)
    
    patients['BIRTHPLACE'] = np.random.choice(birthplaces, size=len(patients), replace=True)

    return patients


# Load, modify, and save the Train data
train_data_path = '../data/Train/csv/patients.csv'
modified_train_patients = load_and_modify(train_data_path)
modified_train_patients.to_csv('../data/Train/csv/patients_australianised.csv', index=False)

# Load, modify, and save the Test data
test_data_path = '../data/Test/csv/patients.csv'
modified_test_patients = load_and_modify(test_data_path)
modified_test_patients.to_csv('../data/Test/csv/patients_australianised.csv', index=False)

print("All patient data processed and saved with Australianised details.")


All patient data processed and saved with Australianised details.
